In [1]:
import requests
from bs4 import BeautifulSoup

def get_webpage_content(url):
    try:
        # 发送HTTP GET请求到指定的URL
        response = requests.get(url)
        
        # 检查请求是否成功（状态码200表示成功）
        if response.status_code == 200:
            # 将网页内容解析为BeautifulSoup对象
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # 返回解析后的内容
            return soup
        else:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None




In [2]:
url = "http://10.15.89.177:8899/gpu"  # 替换为你想要解析的URL
soup = get_webpage_content(url)

if soup:
    print(soup.prettify())  # 打印出网页内容的结构化表示

<!DOCTYPE html>
<html lang="zh_CN">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Docutils 0.17.1: http://docutils.sourceforge.net/" name="generator"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="Copy to clipboard" name="lang:clipboard.copy"/>
  <meta content="Copied to clipboard" name="lang:clipboard.copied"/>
  <meta content="en" name="lang:search.language"/>
  <meta content="True" name="lang:search.pipeline.stopwords"/>
  <meta content="True" name="lang:search.pipeline.trimmer"/>
  <meta content="No matching documents" name="lang:search.result.none"/>
  <meta content="1 matching document" name="lang:search.result.one"/>
  <meta content="# matching documents" name="lang:search.result.other"/>
  <meta content="[\s\-]+" name="lang:search.tokenizer"/>
  <link crossorigin="" href="https://fonts.gstatic.com/" re

In [3]:
def parse_gpu_info(soup):
    results = []
    nodes = soup.find_all('h4')
    
    for node in nodes:
        node_name = node.get_text().split()[0]
        table = node.find_next('table', class_='table table-striped table-condensed')
        if table:
            gpus = table.find_all('tr')[1:]  # Skip the header row
            for gpu in gpus:
                gpu_name = gpu.find_all('td')[1].get_text(strip=True)
                gpu_usage = gpu.find_all('td')[3].find('div', class_='progress-bar')['style'].split(':')[-1].strip()
                
                results.append({
                    'Node Name': node_name,
                    'GPU Model': gpu_name,
                    'GPU Usage': gpu_usage
                })
    
    return results

In [4]:
gpu_info = parse_gpu_info(soup)
for info in gpu_info:
    print(info)

{'Node Name': 'ai_gpu01', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '90%'}
{'Node Name': 'ai_gpu01', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '100%'}
{'Node Name': 'ai_gpu01', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '100%'}
{'Node Name': 'ai_gpu01', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '100%'}
{'Node Name': 'ai_gpu01', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '100%'}
{'Node Name': 'ai_gpu01', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '100%'}
{'Node Name': 'ai_gpu01', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '8%'}
{'Node Name': 'ai_gpu01', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '68%'}
{'Node Name': 'ai_gpu02', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '47%'}
{'Node Name': 'ai_gpu02', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '53%'}
{'Node Name': 'ai_gpu02', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '63%'}
{'Node Name': 'ai_gpu02', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '47%'}
{'Node Name': 'ai_gpu02', 'GPU Model': 'NVIDIA A40', 'GPU Usage': '66%'}
{'Node Name': 'ai_gpu02', 'GPU Model': 'NVIDIA 

In [7]:
from collections import defaultdict
def classify_by_gpu_model(gpu_info):
    classified_info = defaultdict(list)
    
    for info in gpu_info:
        gpu_model = info['GPU Model']
        gpu_usage = info['GPU Usage']
        
        if gpu_usage == '0%':
            classified_info[gpu_model].append(info['Node Name'])
    
    return classified_info


classified_info = classify_by_gpu_model(gpu_info)
   
print("空闲节点列表:")     
for gpu_model, nodes in classified_info.items():
    print(f"GPU Model: {gpu_model}")
    print(f"Nodes with 0% GPU Usage: {', '.join(nodes)}\n")

空闲节点列表:
GPU Model: NVIDIA A40
Nodes with 0% GPU Usage: ai_gpu03, ai_gpu05, ai_gpu05, ai_gpu07, ai_gpu09, ai_gpu10, sist-a40-06, sist-a40-06, sist-a40-06, sist-a40-09, sist-a40-09, sist-a40-09, sist-a40-09, sist-a40-09, sist-a40-09, sist-a40-09

GPU Model: NVIDIA GeForce RTX 2080 Ti
Nodes with 0% GPU Usage: ai_gpu14, ai_gpu14, ai_gpu15, ai_gpu15, ai_gpu17, ai_gpu17, ai_gpu17, ai_gpu18, ai_gpu19, ai_gpu20, ai_gpu20, ai_gpu20, ai_gpu21, ai_gpu21, ai_gpu23, ai_gpu24, ai_gpu24, ai_gpu24, ai_gpu24, sist_gpu67, sist_gpu67, sist_gpu67, sist_gpu67, sist_gpu68, sist_gpu68, sist_gpu68, sist_gpu69, sist_gpu69, sist_gpu69, sist_gpu70, sist_gpu70, sist_gpu70, sist_gpu70, sist_gpu70, sist_gpu70, sist_gpu70

GPU Model: NVIDIA TITAN RTX
Nodes with 0% GPU Usage: ai_gpu26, ai_gpu27, ai_gpu28, ai_gpu28, ai_gpu28, ai_gpu29, ai_gpu29, ai_gpu29, ai_gpu31, ai_gpu31, ai_gpu32, ai_gpu32, ai_gpu32, ai_gpu32, ai_gpu33

GPU Model: NVIDIA TITAN V
Nodes with 0% GPU Usage: sist_gpu48, sist_gpu48, sist_gpu48, sist_gpu